In [2]:
import wandb
import pandas as pd
import os
from tqdm import tqdm

from table_plotter import print_result_table

In [3]:
api = wandb.Api(timeout=600)


In [4]:
# get all runs for a project
runs = api.runs("inchangbaek4907/pcgrl-llm")
len(runs)

195

In [5]:
# Specify cache directory
cache_dir = "./wandb_cache"
os.makedirs(cache_dir, exist_ok=True)

dfs = []
skipped_runs = []  # List to store IDs of skipped runs

for run in tqdm(runs):
    # Skip the run if it's not finished
    if run.state != "finished":
        print(f"Skipping run {run.id} (state: {run.state})")
        skipped_runs.append(run.id)
        continue

    # Define cache filename based on run ID
    cache_file = os.path.join(cache_dir, f"{run.id}.csv")
    
    # Check if cached file exists
    if os.path.exists(cache_file):
        # Load cached DataFrame
        df = pd.read_csv(cache_file)
    else:
        # Retrieve history data for Evaluation/similarity and diversity
        sim = run.history(keys=["Evaluation/similarity", "Evaluation/llm_iteration"])
        div = run.history(keys=["Evaluation/diversity", "Evaluation/llm_iteration"])
        llm_sim = run.history(keys=["Evaluation/llm/similarity", "Evaluation/llm_iteration"])
        llm_div = run.history(keys=["Evaluation/llm/diversity", "Evaluation/llm_iteration"])
        
        # Sequentially merge the DataFrames on "Evaluation/llm_iteration"
        df = pd.merge(sim, div, on="Evaluation/llm_iteration", how="outer")
        df = df.drop(columns=["_step_x", "_step_y"], errors="ignore")
        if 'Evaluation/llm_iteration' in llm_sim.columns:
            df = pd.merge(df, llm_sim, on="Evaluation/llm_iteration", how="outer")
            df = df.drop(columns=["_step_x", "_step_y"], errors="ignore")
        if 'Evaluation/llm_iteration' in llm_div.columns:
            df = pd.merge(df, llm_div, on="Evaluation/llm_iteration", how="outer")
            df = df.drop(columns=["_step_x", "_step_y"], errors="ignore")
            
        # Now, 'df' contains all the merged data based on "Evaluation/llm_iteration"

        df = df.drop(columns=["_step_x", "_step_y"], errors="ignore")

        # Add run config to DataFrame with prefix 'config.'
        for key, value in run.config.items():
            if isinstance(value, list):
                value = ",".join(map(str, value))  # Convert list to comma-separated string
            df[key] = value

        # Filter columns
        key_filter = ['target_character', 'pe', 'branch_factor', 'exp_name', 'evaluator', 'total_iterations', 
                      'reward_feature', 'fewshot', 'problem', 'seed', 'Evaluation/llm_iteration', 
                      'Evaluation/similarity', 'Evaluation/diversity']
        auxiliary_key_filter = ['Evaluation/llm/similarity', 'Evaluation/llm/diversity']
        
        try:
            df = df[key_filter + auxiliary_key_filter]
        except KeyError:
            df = df[key_filter]
        
        # Save DataFrame to cache as CSV
        df.to_csv(cache_file, index=False)
    
    # Append DataFrame to list
    dfs.append(df)

# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df.head()  # Display the first few rows for inspection

# Print summary of skipped runs
print("\nSummary of Skipped Runs:")
print(f"Total skipped runs: {len(skipped_runs)}")
print("Skipped run IDs:", skipped_runs)

100%|██████████| 195/195 [07:17<00:00,  2.24s/it]


Summary of Skipped Runs:
Total skipped runs: 0
Skipped run IDs: []


In [6]:
time_str = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")

In [7]:
df.to_csv(f"wandb_output_{time_str}.csv", index=False)

In [8]:

df['seed'] = df['seed'] % 3
df

,target_character,pe,branch_factor,exp_name,evaluator,total_iterations,reward_feature,fewshot,problem,seed,Evaluation/llm_iteration,Evaluation/similarity,Evaluation/diversity,Evaluation/llm/similarity,Evaluation/llm/diversity
0,N,io,2,def,vit,1,array,False,binary,1,1,0.018449,0.595029,NaN,NaN
1,M,io,2,def,vit,1,array,False,binary,1,1,0.045062,0.147950,NaN,NaN
2,F,io,2,def,vit,1,array,False,binary,1,1,0.799518,0.000000,NaN,NaN
3,D,io,2,def,vit,1,array,False,binary,1,1,0.032404,0.137656,NaN,NaN
4,C,io,2,def,vit,1,array,False,binary,1,1,0.765589,0.000347,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,M,got,2,longiter,vit,9,array,False,binary,0,5,0.049230,0.167293,NaN,NaN
1123,M,got,2,longiter,vit,9,array,False,binary,0,6,0.051012,0.167400,NaN,NaN
1124,M,got,2,longiter,vit,9,array,False,binary,0,7,0.202214,0.204327,NaN,NaN
1125,M,got,2,longiter,vit,9,array,False,binary,0,8,0.608841,0.016238,NaN,NaN


In [9]:
# remove pe == cotsc
df = df[df['pe'] != 'cotsc']

# if the exname is not def make the pe to 'pe+exp_name'
df['pe'] = df.apply(lambda x: x['pe'] + '-' +  x['exp_name'] if x['exp_name'] != 'def' else x['pe'], axis=1)

In [10]:
df.groupby(['pe', 'evaluator', 'fewshot', 'seed']).count() 

target_character  branch_factor  \
pe           evaluator fewshot seed                                    
cot          vit       False   0                   24             24   
                               1                   36             36   
                               2                   30             30   
                       True    0                   30             30   
                               1                   30             30   
                               2                   30             30   
cot-encfb    vit       False   0                   30             30   
                               1                   30             30   
                               2                   29             29   
got          llm       False   0                   30             30   
                               1                   30             30   
                               2                   30             30   
             vit       False   0                   30             30   
                               1                   30             30   
                               2                   25             25   
                       True    0                   30             30   
                               1                   30             30   
                               2                   30             30   
got-encfb    vit       False   0                   28             28   
                               1                   25             25   
                               2                   30             30   
got-longiter vit       False   0                   45             45   
                               1                   45             45   
                               2                   45             45   
io           vit       False   0                    5              5   
                               1                    5              5   
                               2                    5              5   
tot          llm       False   0                   30             30   
                               1                   30             30   
                               2                   30             30   
             vit       False   0                   30             30   
                               1                   30             30   
                               2                   30             30   
                       True    0                   30             30   
                               1                   30             30   
                               2                   30             30   
tot-encfb    vit       False   0                   30             30   
                               1                   30             30   
                               2                   30             30   

                                     exp_name  total_iterations  \
pe           evaluator fewshot seed                               
cot          vit       False   0           24                24   
                               1           36                36   
                               2           30                30   
                       True    0           30                30   
                               1           30                30   
                               2           30                30   
cot-encfb    vit       False   0           30                30   
                               1           30                30   
                               2           29                29   
got          llm       False   0           30                30   
                               1           30                30   
                               2           30                30   
             vit       False   0           30                30   
                               1           30                30   
            

In [11]:
# Min-max Normalize Evaluation/similarity and Evaluation/diversity for each 'target_character' and add a new column 'Evaluation/similarity/norm' and 'Evaluation/diversity/norm'
df['Evaluation/similarity/norm'] = df.groupby('target_character')['Evaluation/similarity'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df['Evaluation/diversity/norm'] = df.groupby('target_character')['Evaluation/diversity'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df

,target_character,pe,branch_factor,exp_name,evaluator,total_iterations,reward_feature,fewshot,problem,seed,Evaluation/llm_iteration,Evaluation/similarity,Evaluation/diversity,Evaluation/llm/similarity,Evaluation/llm/diversity,Evaluation/similarity/norm,Evaluation/diversity/norm
0,N,io,2,def,vit,1,array,False,binary,1,1,0.018449,0.595029,NaN,NaN,0.010878,1.000000
1,M,io,2,def,vit,1,array,False,binary,1,1,0.045062,0.147950,NaN,NaN,0.045574,0.315946
2,F,io,2,def,vit,1,array,False,binary,1,1,0.799518,0.000000,NaN,NaN,0.950696,0.000000
3,D,io,2,def,vit,1,array,False,binary,1,1,0.032404,0.137656,NaN,NaN,0.034002,0.247650
4,C,io,2,def,vit,1,array,False,binary,1,1,0.765589,0.000347,NaN,NaN,0.947774,0.000649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,M,got-longiter,2,longiter,vit,9,array,False,binary,0,5,0.049230,0.167293,NaN,NaN,0.050856,0.357254
1123,M,got-longiter,2,longiter,vit,9,array,False,binary,0,6,0.051012,0.167400,NaN,NaN,0.053114,0.357481
1124,M,got-longiter,2,longiter,vit,9,array,False,binary,0,7,0.202214,0.204327,NaN,NaN,0.244729,0.436339
1125,M,got-longiter,2,longiter,vit,9,array,False,binary,0,8,0.608841,0.016238,NaN,NaN,0.760041,0.034675


In [12]:
# 각 실험마다 마지막 Iteration의 Similarity와 Diversity를 가져옴, 또한 각 실험의 최대 Similarity와 최소 Diversity를 가져옴
# Update the function to include calculations based on normalized similarity and diversity
experiment_cols = ['target_character', 'pe', 'evaluator', 'fewshot', 'seed']  # Assuming these are the experiment identifiers as per user's requirement
experiment_cols_without_seed = ['target_character', 'pe', 'evaluator', 'fewshot']

In [13]:

def extract_metrics_with_norm(group):
    # Maximum similarity iteration row
    max_sim_row = group.loc[group['Evaluation/similarity'].idxmax()]
    max_sim_norm_row = group.loc[group['Evaluation/similarity/norm'].idxmax()]
    # Final iteration row
    final_iteration_row = group.loc[group['Evaluation/llm_iteration'].idxmax()]
    
    return pd.Series({
        'Max Similarity Iteration': max_sim_row['Evaluation/llm_iteration'],
        'Max Similarity': max_sim_row['Evaluation/similarity'],
        'Diversity at Max Similarity': max_sim_row['Evaluation/diversity'],
        
        'Max Normalized Similarity Iteration': max_sim_norm_row['Evaluation/llm_iteration'],
        'Max Normalized Similarity': max_sim_norm_row['Evaluation/similarity/norm'],
        'Diversity at Max Normalized Similarity': max_sim_norm_row['Evaluation/diversity/norm'],
        
        'Final Iteration': final_iteration_row['Evaluation/llm_iteration'],
        'Final Similarity': final_iteration_row['Evaluation/similarity'],
        'Final Diversity': final_iteration_row['Evaluation/diversity'],
        'Final Normalized Similarity': final_iteration_row['Evaluation/similarity/norm'],
        'Final Normalized Diversity': final_iteration_row['Evaluation/diversity/norm']
    })

# Apply the updated function to each experiment group
result = df.groupby(experiment_cols).apply(extract_metrics_with_norm).reset_index()

# Display the enhanced results to the user
result

/var/folders/x_/2lt9k5kn52q43m1_z0kp7tfm0000gn/T/ipykernel_25115/2537490999.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(experiment_cols).apply(extract_metrics_with_norm).reset_index()


,target_character,pe,evaluator,fewshot,seed,Max Similarity Iteration,Max Similarity,Diversity at Max Similarity,Max Normalized Similarity Iteration,Max Normalized Similarity,Diversity at Max Normalized Similarity,Final Iteration,Final Similarity,Final Diversity,Final Normalized Similarity,Final Normalized Diversity
0,C,cot,vit,False,1,5.0,0.199735,0.068222,5.0,0.238671,0.127580,6.0,0.017137,0.226798,0.009846,0.424131
1,C,cot,vit,False,2,3.0,0.165821,0.000000,3.0,0.196170,0.000000,6.0,0.019047,0.124546,0.012240,0.232912
2,C,cot,vit,True,0,1.0,0.171087,0.022992,1.0,0.202770,0.042997,6.0,0.171087,0.022992,0.202770,0.042997
3,C,cot,vit,True,1,6.0,0.483973,0.026739,6.0,0.594866,0.050004,6.0,0.483973,0.026739,0.594866,0.050004
4,C,cot,vit,True,2,4.0,0.239788,0.126869,4.0,0.288863,0.237256,6.0,0.129345,0.016808,0.150460,0.031432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,N,tot,vit,True,1,1.0,0.048219,0.081245,1.0,0.047675,0.136539,6.0,0.027921,0.000000,0.022586,0.000000
190,N,tot,vit,True,2,2.0,0.040261,0.260393,2.0,0.037839,0.437613,6.0,0.026634,0.124258,0.020995,0.208827
191,N,tot-encfb,vit,False,0,5.0,0.811480,0.000000,5.0,0.991102,0.000000,6.0,0.807737,0.000029,0.986475,0.000049
192,N,tot-encfb,vit,False,1,1.0,0.269528,0.000000,1.0,0.321224,0.000000,6.0,0.027921,0.000000,0.022586,0.000000


In [14]:
plotter = {'Max Normalized Similarity': ['mean', 'std'], 'Diversity at Max Normalized Similarity': ['mean', 'std'], 'Max Similarity Iteration': ['mean', 'std'], 'Final Normalized Similarity': ['mean', 'std'], 'Final Normalized Diversity': ['mean', 'std']}

In [16]:
result.groupby(['pe', 'evaluator', 'fewshot', 'target_character']).agg(plotter)

Max Normalized Similarity  \
                                                                  mean   
pe        evaluator fewshot target_character                             
cot       vit       False   C                                 0.217420   
                            D                                 0.079968   
                            F                                 0.872418   
                            M                                 0.275086   
                            N                                 0.679753   
...                                                                ...   
tot-encfb vit       False   C                                 0.743260   
                            D                                 0.287982   
                            F                                 0.873448   
                            M                                 0.305698   
                            N                                 0.463715   

                                                        \
                                                   std   
pe        evaluator fewshot target_character             
cot       vit       False   C                 0.030053   
                            D                 0.056442   
                            F                 0.174582   
                            M                 0.377989   
                            N                 0.546996   
...                                                ...   
tot-encfb vit       False   C                 0.383682   
                            D                 0.164980   
                            F                 0.138057   
                            M                 0.451319   
                            N                 0.472538   

                                             Diversity at Max Normalized Similarity  \
                                                                               mean   
pe        evaluator fewshot target_character                                          
cot       vit       False   C                                              0.063790   
                            D                                              0.186997   
                            F                                              0.003149   
                            M                                              0.329053   
                            N                                              0.090928   
...                                                                             ...   
tot-encfb vit       False   C                                              0.056582   
                            D                                              0.048940   
                            F                                              0.007603   
                            M                                              0.059236   
                            N                                              0.177124   

                                                        \
                                                   std   
pe        evaluator fewshot target_character             
cot       vit       False   C                 0.090213   
                            D                 0.180431   
                            F                 0.004980   
                            M                 0.285080   
                            N                 0.157485   
...                                                ...   
tot-encfb vit       False   C                 0.097812   
                            D                 0.083838   
                            F                 0.012478   
                            M                 0.099586   
                            N                 0.306787   

                                             Max Similarity Iteration  \
                                                                 mean   
pe        evaluat

## Exp 1. Reasoning performance of prompt engineering

In [17]:
# Get the fewshot=False and evaluator=vit
exp1_df = result[(result['fewshot'] == False) & (result['evaluator'] == 'vit')]
exp1_df.groupby(experiment_cols_without_seed).count()

seed  \
target_character pe           evaluator fewshot         
C                cot          vit       False       2   
                 cot-encfb    vit       False       3   
                 got          vit       False       3   
                 got-encfb    vit       False       3   
                 got-longiter vit       False       3   
                 io           vit       False       3   
                 tot          vit       False       3   
                 tot-encfb    vit       False       3   
D                cot          vit       False       3   
                 cot-encfb    vit       False       3   
                 got          vit       False       3   
                 got-encfb    vit       False       3   
                 got-longiter vit       False       3   
                 io           vit       False       3   
                 tot          vit       False       3   
                 tot-encfb    vit       False       3   
F                cot          vit       False       3   
                 cot-encfb    vit       False       3   
                 got          vit       False       3   
                 got-encfb    vit       False       3   
                 got-longiter vit       False       3   
                 io           vit       False       3   
                 tot          vit       False       3   
                 tot-encfb    vit       False       3   
M                cot          vit       False       3   
                 cot-encfb    vit       False       3   
                 got          vit       False       3   
                 got-encfb    vit       False       3   
                 got-longiter vit       False       3   
                 io           vit       False       3   
                 tot          vit       False       3   
                 tot-encfb    vit       False       3   
N                cot          vit       False       3   
                 cot-encfb    vit       False       3   
                 got          vit       False       3   
                 got-encfb    vit       False       3   
                 got-longiter vit       False       3   
                 io           vit       False       3   
                 tot          vit       False       3   
                 tot-encfb    vit       False       3   

                                                 Max Similarity Iteration  \
target_character pe           evaluator fewshot                             
C                cot          vit       False                           2   
                 cot-encfb    vit       False                           3   
                 got          vit       False                           3   
                 got-encfb    vit       False                           3   
                 got-longiter vit       False                           3   
                 io           vit       False                           3   
                 tot          vit       False                           3   
                 tot-encfb    vit       False                           3   
D                cot          vit       False                           3   
                 cot-encfb    vit       False                           3   
                 got          vit       False                           3   
                 got-encfb    vit       False                           3   
                 got-longiter vit       False                           3   
                 io           vit       False                           3   
                 tot          vit       False                           3   
                 tot-encfb    vit       False                           3   
F                cot          vit       False                           3   
                 cot-encfb    vit       False                           3   
                 got          vit       False                           3   
                 got-encfb    vit    

In [18]:
exp1_df.groupby(experiment_cols_without_seed).agg(plotter)

Max Normalized Similarity  \
                                                                     mean   
target_character pe           evaluator fewshot                             
C                cot          vit       False                    0.217420   
                 cot-encfb    vit       False                    0.442607   
                 got          vit       False                    0.383675   
                 got-encfb    vit       False                    0.216658   
                 got-longiter vit       False                    0.815252   
                 io           vit       False                    0.338068   
                 tot          vit       False                    0.329980   
                 tot-encfb    vit       False                    0.743260   
D                cot          vit       False                    0.079968   
                 cot-encfb    vit       False                    0.144626   
                 got          vit       False                    0.527393   
                 got-encfb    vit       False                    0.316793   
                 got-longiter vit       False                    0.538999   
                 io           vit       False                    0.112471   
                 tot          vit       False                    0.444530   
                 tot-encfb    vit       False                    0.287982   
F                cot          vit       False                    0.872418   
                 cot-encfb    vit       False                    0.912394   
                 got          vit       False                    0.628293   
                 got-encfb    vit       False                    0.757300   
                 got-longiter vit       False                    0.730584   
                 io           vit       False                    0.321133   
                 tot          vit       False                    0.895790   
                 tot-encfb    vit       False                    0.873448   
M                cot          vit       False                    0.275086   
                 cot-encfb    vit       False                    0.592049   
                 got          vit       False                    0.354410   
                 got-encfb    vit       False                    0.707481   
                 got-longiter vit       False                    0.648403   
                 io           vit       False                    0.188986   
                 tot          vit       False                    0.119982   
                 tot-encfb    vit       False                    0.305698   
N                cot          vit       False                    0.679753   
                 cot-encfb    vit       False                    0.162077   
                 got          vit       False                    0.179544   
                 got-encfb    vit       False                    0.374978   
                 got-longiter vit       False                    0.391305   
                 io           vit       False                    0.045186   
                 tot          vit       False                    0.664377   
                 tot-encfb    vit       False                    0.463715   

                                                           \
                                                      std   
target_character pe           evaluator fewshot             
C                cot          vit       False    0.030053   
                 cot-encfb    vit       False    0.216077   
                 got          vit       False    0.535422   
                 got-encfb    vit       False    0.022243   
                 got-longiter vit       False    0.023105   
                 io           vit       False    0.528421   
                 tot          vit       False    0.259856   
                 tot-encfb    vit       False    0.383682   
D                cot          vit       False    0.056442   
    

In [19]:
print_result_table(exp1_df, category_columns=['pe', 'target_character'])

AttributeError: 'tuple' object has no attribute 'split'

## Exp. 2: LLM and ViT-based node evaluation 

In [18]:
exp2_df = result[(result['fewshot'] == False)]
exp2_df = exp2_df[exp2_df['pe'].isin(['tot', 'got'])]
exp2_df.groupby(experiment_cols_without_seed).count()

seed  Max Similarity Iteration  Max Similarity  \
pe  evaluator fewshot                                                   
got llm       False       3                         3               3   
    vit       False       3                         3               3   
tot llm       False       3                         3               3   
    vit       False       3                         3               3   

                       Diversity at Max Similarity  \
pe  evaluator fewshot                                
got llm       False                              3   
    vit       False                              3   
tot llm       False                              3   
    vit       False                              3   

                       Max Normalized Similarity Iteration  \
pe  evaluator fewshot                                        
got llm       False                                      3   
    vit       False                                      3   
tot llm       False                                      3   
    vit       False                                      3   

                       Max Normalized Similarity  \
pe  evaluator fewshot                              
got llm       False                            3   
    vit       False                            3   
tot llm       False                            3   
    vit       False                            3   

                       Diversity at Max Normalized Similarity  \
pe  evaluator fewshot                                           
got llm       False                                         3   
    vit       False                                         3   
tot llm       False                                         3   
    vit       False                                         3   

                       Final Iteration  Final Similarity  Final Diversity  \
pe  evaluator fewshot                                                       
got llm       False                  3                 3                3   
    vit       False                  3                 3                3   
tot llm       False                  3                 3                3   
    vit       False                  3                 3                3   

                       Final Normalized Similarity  Final Normalized Diversity  
pe  evaluator fewshot                                                           
got llm       False                              3                           3  
    vit       False                              3                           3  
tot llm       False                              3                           3  
    vit       False                              3                           3

In [19]:
# get only cot and tot
exp2_df.groupby(experiment_cols_without_seed).agg(plotter)

Max Normalized Similarity            \
                                           mean       std   
pe  evaluator fewshot                                       
got llm       False                    0.942574  0.060667   
    vit       False                    0.937551  0.067060   
tot llm       False                    0.880511  0.178100   
    vit       False                    0.987594  0.014482   

                      Diversity at Max Normalized Similarity            \
                                                        mean       std   
pe  evaluator fewshot                                                    
got llm       False                                 0.000477  0.000739   
    vit       False                                 0.002276  0.003942   
tot llm       False                                 0.086572  0.149927   
    vit       False                                 0.000018  0.000031   

                      Max Similarity Iteration            \
                                          mean       std   
pe  evaluator fewshot                                      
got llm       False                   1.666667  1.154701   
    vit       False                   2.666667  2.081666   
tot llm       False                   2.000000  1.000000   
    vit       False                   4.000000  2.645751   

                      Final Normalized Similarity            \
                                             mean       std   
pe  evaluator fewshot                                         
got llm       False                      0.021929  0.014777   
    vit       False                      0.188936  0.196033   
tot llm       False                      0.423527  0.422219   
    vit       False                      0.341474  0.465530   

                      Final Normalized Diversity            
                                            mean       std  
pe  evaluator fewshot                                       
got llm       False                     0.147448  0.227586  
    vit       False                     0.162179  0.233817  
tot llm       False                     0.060853  0.105401  
    vit       False                     0.447746  0.407723

In [20]:
print_result_table(exp2_df, category_columns=['evaluator', 'pe'])

AttributeError: 'tuple' object has no attribute 'split'

## Exp 3: Controlling the first iteration reward function

In [21]:
# Get the fewshot=True and evaluator=vit
exp3_df = result[(result['pe'] != 'io') & (result['evaluator'] == 'vit')]
exp3_df.groupby(experiment_cols_without_seed).count()

seed  Max Similarity Iteration  \
pe           evaluator fewshot                                   
cot          vit       False       3                         3   
                       True        3                         3   
cot-encfb    vit       False       3                         3   
got          vit       False       3                         3   
                       True        3                         3   
got-encfb    vit       False       3                         3   
got-longiter vit       False       3                         3   
tot          vit       False       3                         3   
                       True        3                         3   
tot-encfb    vit       False       3                         3   

                                Max Similarity  Diversity at Max Similarity  \
pe           evaluator fewshot                                                
cot          vit       False                 3                            3   
                       True                  3                            3   
cot-encfb    vit       False                 3                            3   
got          vit       False                 3                            3   
                       True                  3                            3   
got-encfb    vit       False                 3                            3   
got-longiter vit       False                 3                            3   
tot          vit       False                 3                            3   
                       True                  3                            3   
tot-encfb    vit       False                 3                            3   

                                Max Normalized Similarity Iteration  \
pe           evaluator fewshot                                        
cot          vit       False                                      3   
                       True                                       3   
cot-encfb    vit       False                                      3   
got          vit       False                                      3   
                       True                                       3   
got-encfb    vit       False                                      3   
got-longiter vit       False                                      3   
tot          vit       False                                      3   
                       True                                       3   
tot-encfb    vit       False                                      3   

                                Max Normalized Similarity  \
pe           evaluator fewshot                              
cot          vit       False                            3   
                       True                             3   
cot-encfb    vit       False                            3   
got          vit       False                            3   
                       True                             3   
got-encfb    vit       False                            3   
got-longiter vit       False                            3   
tot          vit       False                            3   
                       True                             3   
tot-encfb    vit       False                            3   

                                Diversity at Max Normalized Similarity  \
pe           evaluator fewshot                                           
cot          vit       False                                         3   
                       True                                          3   
cot-encfb    vit       False                                         3   
got          vit       False                                         3   
                       True                                          3   
got-encfb    vit       False                                         3   
got-longiter vit       False                                         3   
tot          vit       False          

In [22]:
exp3_df.groupby(experiment_cols_without_seed).agg(plotter)

Max Normalized Similarity            \
                                                    mean       std   
pe           evaluator fewshot                                       
cot          vit       False                    0.982591  0.022884   
                       True                     0.913924  0.052203   
cot-encfb    vit       False                    0.912394  0.076618   
got          vit       False                    0.937551  0.067060   
                       True                     0.929675  0.055938   
got-encfb    vit       False                    0.986836  0.009218   
got-longiter vit       False                    0.885514  0.099275   
tot          vit       False                    0.987594  0.014482   
                       True                     0.921507  0.065687   
tot-encfb    vit       False                    0.967582  0.020409   

                               Diversity at Max Normalized Similarity  \
                                                                 mean   
pe           evaluator fewshot                                          
cot          vit       False                                 0.000153   
                       True                                  0.001135   
cot-encfb    vit       False                                 0.003670   
got          vit       False                                 0.002276   
                       True                                  0.003333   
got-encfb    vit       False                                 0.000002   
got-longiter vit       False                                 0.085865   
tot          vit       False                                 0.000018   
                       True                                  0.000145   
tot-encfb    vit       False                                 0.000292   

                                         Max Similarity Iteration            \
                                     std                     mean       std   
pe           evaluator fewshot                                                
cot          vit       False    0.000258                 4.000000  1.732051   
                       True     0.001882                 3.000000  2.645751   
cot-encfb    vit       False    0.005934                 2.666667  0.577350   
got          vit       False    0.003942                 2.666667  2.081666   
                       True     0.005544                 2.333333  2.309401   
got-encfb    vit       False    0.000003                 1.333333  0.577350   
got-longiter vit       False    0.145016                 4.000000  3.000000   
tot          vit       False    0.000031                 4.000000  2.645751   
                       True     0.000251                 1.000000  0.000000   
tot-encfb    vit       False    0.000334                 4.666667  0.577350   

                               Final Normalized Similarity            \
                                                      mean       std   
pe           evaluator fewshot                                         
cot          vit       False                      0.346647  0.554319   
                       True                       0.334658  0.552904   
cot-encfb    vit       False                      0.032899  0.050132   
got          vit       False                      0.188936  0.196033   
                       True                       0.258092  0.389669   
got-encfb    vit       False                      0.358125  0.320487   
got-longiter vit       False                      0.233946  0.179967   
tot          vit       False                      0.341474  0.465530   
                       True                       0.035927  0.049807   
tot-encfb    vit       False                      0.032472  0.018474   

                               Final Normalized Diversity            
                                                     mean       std  
pe           evaluator fewshot                                   

In [23]:
# Short the dataframe with fewshot True is on top
exp3_df = exp3_df.sort_values(by='fewshot', ascending=True)
print_result_table(exp3_df, category_columns=['pe', 'fewshot'])

## Appendix: Additional Analysis